# From the web

In [14]:
import re
from string import maketrans

pattern = re.compile(r'(?=(ATG(?:...)*?)(?=TAG|TGA|TAA))')

def revcomp(dna_seq):
    return dna_seq[::-1].translate(maketrans("ATGC","TACG"))

def orfs(dna):
    return set(pattern.findall(dna) + pattern.findall(revcomp(dna)))

print( orfs(seq))

hej


ImportError: cannot import name 'maketrans'

In [29]:


table = 1
min_pro_len = 100

for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
    for frame in range(3):
        for pro in nuc[frame:].translate(table).split("*"):
            if len(pro) >= min_pro_len:
                print("%s...%s - length %i, strand %i, frame %i" % (pro[:30], pro[-3:], len(pro), strand, frame))


RVMATKAVCVLKGDGPVQGIINFEQKESNG...IAQ - length 156, strand 1, frame 2
GASDYIQGNVYWAIPITPQAKRLPAFPVFV...GLK - length 101, strand -1, frame 0


/home/mattias/anaconda3/envs/itba-bio/lib/python3.6/site-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [33]:
from Bio import SeqIO

record = SeqIO.read("files/ej1/SOD1_mrna.gb", "genbank")
table = 1
min_pro_len = 100


def find_orfs_with_trans(seq, trans_table, min_protein_length):
    answer = []
    seq_len = len(seq)
    for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
        for frame in range(3):
            trans = nuc[frame:].translate(trans_table)
            trans_len = len(trans)
            aa_start = 0
            aa_end = 0
            while aa_start < trans_len:
                aa_end = trans.find("*", aa_start)
                if aa_end == -1:
                    aa_end = trans_len
                if aa_end - aa_start >= min_protein_length:
                    if strand == 1:
                        start = frame + aa_start * 3
                        end = min(seq_len, frame + aa_end * 3 + 3)
                    else:
                        start = seq_len - frame - aa_end * 3 - 3
                        end = seq_len - frame - aa_start * 3
                    answer.append((start, end, strand, trans[aa_start:aa_end]))
                aa_start = aa_end + 1
    answer.sort()
    return answer


orf_list = find_orfs_with_trans(record.seq, table, min_pro_len)
for start, end, strand, pro in orf_list:
    print(
        "%s...%s - length %i, strand %i, %i:%i"
        % (pro[:30], pro[-3:], len(pro), strand, start, end)
    )

RVMATKAVCVLKGDGPVQGIINFEQKESNG...IAQ - length 156, strand 1, 71:542
GASDYIQGNVYWAIPITPQAKRLPAFPVFV...GLK - length 101, strand -1, 265:571


/home/mattias/anaconda3/envs/itba-bio/lib/python3.6/site-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


# Own functions

In [68]:
sequence_full = list(SeqIO.parse("files/ej1/SOD1_mrna.gb", "genbank"))[0]

reading_frames = createReadingFrames(sequence_full)


for key, sequence in reading_frames.items():
    print(sequence)
    print(len(sequence)%3)
    print()

ID: NM_000454.5
Name: NM_000454
Description: Homo sapiens superoxide dismutase 1 (SOD1), mRNA
Number of features: 17
/molecule_type=mRNA
Seq('GCGTCGTAGTCTCCTGCAGCGTCTGGGGTTTCCGTTGCAGTCCTCGGAACCAGG...TAA')
0

ID: NM_000454.5
Name: NM_000454
Description: Homo sapiens superoxide dismutase 1 (SOD1), mRNA
Number of features: 18
/molecule_type=mRNA
Seq('CGTCGTAGTCTCCTGCAGCGTCTGGGGTTTCCGTTGCAGTCCTCGGAACCAGGA...AAA')
0

ID: NM_000454.5
Name: NM_000454
Description: Homo sapiens superoxide dismutase 1 (SOD1), mRNA
Number of features: 16
/molecule_type=mRNA
Seq('GTCGTAGTCTCCTGCAGCGTCTGGGGTTTCCGTTGCAGTCCTCGGAACCAGGAC...CTA')
0

ID: NM_000454.5
Name: NM_000454
Description: Homo sapiens superoxide dismutase 1 (SOD1), mRNA
Number of features: 0
/molecule_type=mRNA
Seq('AAATCAAACTTAAACCTAAGAAAATTATCGGAGTATTATTCACGGTATGTCCCA...TGC')
0

ID: NM_000454.5
Name: NM_000454
Description: Homo sapiens superoxide dismutase 1 (SOD1), mRNA
Number of features: 0
/molecule_type=mRNA
Seq('AATCAAACTTAAACCTAAGAAAATTATC

In [66]:
# Returns the 6 reading frames in a dictionary
def createReadingFrames(sequence_full):
    # Create reading frames
    reading_frames = {}

    # Create forward reading frames
    for i in range(3):
        reading_frames[i+1] = sequence_full[i:]

    # Create reverse reading frames
    sequence_full_reverse = sequence_full[::-1]
    for i in range(3):
        reading_frames[-(i+1)] = sequence_full_reverse[i:]

    # Ensure divisible by 3 by truncation
    for key, sequence in reading_frames.items():
            n_extra_nucleotides = len(sequence) % 3
            if n_extra_nucleotides != 0:
                reading_frames[key] = sequence[:-n_extra_nucleotides]

    return reading_frames

In [22]:
def findORF(sequence, n_codons_min):
# Add complements?
    codons_start_RNA = ["AUG"]
    codons_start_DNA = ["ATG"]
    codons_start = codons_start_RNA + codons_start_DNA

    codons_stop_RNA = ["UAG", "UAA", "UGA"]
    codons_stop_DNA = ["TAG", "TAA", "TGA", "GAC"]
    codons_stop = codons_stop_RNA + codons_stop_DNA


# Split by stop codons
    ORF_indices_list = []
    idx_split_start = 0
    for i in range(0, len(sequence), 3):
        codon = str(sequence[i:i+3])
        
        # Check if stop codon
        if codon in codons_stop or i == len(sequence)-3:
            idx_split_end = i
            
            # Skip if split is below minumum length
            split_length = (idx_split_end -idx_split_start)/3
            if split_length < n_codons_min:
                idx_split_start = idx_split_end + 3
                continue

            # Check for orfs by finding start codons
            for j in range(idx_split_start, idx_split_end, 3):
                codon_orf = str(sequence[j:j+3])

                if codon_orf in codons_start:
                    idx_start = j
                    idx_end = idx_split_end -3 # -3 since should not include end codon
                    indices = (idx_start, idx_end)
                    codon_length = (idx_end - idx_start)/3
                    
                    if codon_length >= n_codons_min:
                        print(f"start: {j}, end codon (not included): {idx_split_end} with length {codon_length}")
                        print()
                        ORF_indices_list.append(indices)
                    
            # New start index
            idx_split_start = idx_split_end
            
    return ORF_indices_list



# Search each split for start codons

In [56]:
sequence_full.seq

Seq('GCGTCGTAGTCTCCTGCAGCGTCTGGGGTTTCCGTTGCAGTCCTCGGAACCAGG...AAA')

In [70]:
seq_test_1 = "GCCATTCTTGCCATTCTTAUGGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTTAGGCCATTCTTGCCATTCTT"

seq_test_2 = "GCCATTCTTGCCATTCTTAUGGCCATTCTTGCCAUGATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTGCCATTCTTTAGAUGGCCATTCTTGCCATTCTT"

seq_test_3 = "GCCTAGGCCTAGGCCTAGGCCGCCTAGGCCGCC"

findOrf(seq_test_2, 1)

start: 18, end codon (not included): 105 with length 28.0

start: 33, end codon (not included): 105 with length 23.0

start: 108, end codon (not included): 126 with length 5.0



[(18, 102), (33, 102), (108, 123)]

In [75]:
# sequence = reading_frames[1]
n_codons_min = 10 

for key, sequence in reading_frames.items():
    print(f"frame: {key}")
    # print(sequence)
    # print(len(sequence))
    findOrf(sequence.seq, n_codons_min)
    print()

frame: 1
start: 207, end codon (not included): 339 with length 43.0

start: 216, end codon (not included): 339 with length 40.0

start: 222, end codon (not included): 339 with length 38.0

start: 306, end codon (not included): 339 with length 10.0

start: 354, end codon (not included): 387 with length 10.0

start: 438, end codon (not included): 522 with length 27.0

start: 450, end codon (not included): 522 with length 23.0

start: 471, end codon (not included): 522 with length 16.0

start: 789, end codon (not included): 861 with length 23.0


frame: 2

frame: 3
start: 75, end codon (not included): 108 with length 10.0

start: 846, end codon (not included): 888 with length 13.0


frame: -1

frame: -2
start: 294, end codon (not included): 333 with length 12.0

start: 411, end codon (not included): 444 with length 10.0

start: 738, end codon (not included): 774 with length 11.0


frame: -3



In [78]:
orf = "ATGGCGACGAAGGCCGTGTGCGTGCTGAAGGGCGACGGCCCAGTGCAGGGCATCATCAATTTCGAGCAGAAGGAAAGTAATGGACCAGTGAAGGTGTGGGGAAGCATTAAAGGACTGACTGAAGGCCTGCATGGATTCCATGTTCATGAGTTTGGAGATAATACAGCAGGCTGTACCAGTGCAGGTCCTCACTTTAATCCTCTATCCAGAAAACACGGTGGGCCAAAGGATGAAGAGAGGCATGTTGGAGACTTGGGCAATGTGACTGCTGACAAAGATGGTGTGGCCGATGTGTCTATTGAAGATTCTGTGATCTCACTCTCAGGAGACCATTGCATCATTGGCCGCACACTGGTGGTCCATGAAAAAGCAGATGACTTGGGCAAAGGTGGAAATGAAGAAAGTACAAAGACAGGAAACGCTGGAAGTCGTTTGGCTTGTGGTGTAATTGGGATCGCCCAATAA"

codons_start_RNA = ["AUG"]
codons_start_DNA = ["ATG"]
codons_start = codons_start_RNA + codons_start_DNA

codons_stop_RNA = ["UAG", "UAA", "UGA"]
codons_stop_DNA = ["TAG", "TAA", "TGA", "GAC"]
codons_stop = codons_stop_RNA + codons_stop_DNA

sequence = reading_frames[-1].seq

for i in range(0, len(orf), 3):
    codon = str(sequence[i:i+3])
    if codon in codons_start:
        print(f"idx:{i}, start codon: {codon}")

    if codon in codons_stop:
        print(f"idx:{i}, stop codon: {codon}")

idx:99, start codon: ATG
idx:111, stop codon: TAA
idx:210, start codon: ATG
idx:216, start codon: ATG
idx:228, stop codon: TGA
idx:231, stop codon: GAC
idx:249, start codon: ATG
idx:252, stop codon: TGA
idx:261, stop codon: TAA
idx:336, start codon: ATG
idx:339, stop codon: TAG
idx:363, stop codon: TAG
idx:369, stop codon: TAA
